In [2]:
import pyvisa as visa
import time
import numpy as np
import pymeasure
from pymeasure.instruments.keithley import Keithley2400
import pandas as pd
import matplotlib.pyplot as plt
import datetime


def nano_change_chanel(volt, chan=None):
    if not chan:
        return
    volt.write(":SENS:CHAN {}".format(chan))
    time.sleep(0.1)
    if volt.query(":SENS:CHAN?").strip() == str(chan):
        return
    else:
        print("channel switch error")
        raise

def init_hall_two_channel(dc_current=50e-6, step=3):
    for i in range(step):
        print("sample 1:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc1, volt1, dc_current=dc_current))
    print("-"*10)
    for i in range(step):
        print("sample 2:" + str(i) + "chan1, std_1, chan_2,std_2")
        print(measure_resistance_two_channel(dc2, volt2, dc_current=dc_current))
        
def measure_resistance(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.1):
    dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    dc.compliance_voltage = 10        # Sets the compliance voltage to 10 V
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up = []
    for i in range(30):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up.append(float(volt.query("fetch?")))
    v_up = sorted(v_up)
    v_up = v_up[3: -3]
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down = []
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down.append(float(volt.query("fetch?")))
    
    dc.shutdown() 
    #dc.source_current = dc_current
    #dc.enable_source() 
    
    v_down = sorted(v_down)
    v_down = v_down[3: -3]
    
    average_v = (np.array(v_up).mean() - np.array(v_down).mean())/2
    std_v = (np.array(v_up).std() + np.array(v_down).std())/2
    return average_v, std_v


def measure_resistance_two_channel(dc, volt, dc_current=100e-6, measure_points=30, skip_points=10, time_per_point=0.05):
    #dc.apply_current()  # Sets up to source current
    dc.source_current_range = 10e-3   # Sets the source current range to 10 mA
    
    dc.compliance_voltage = 100        # Sets the compliance voltage to 100 V
    
    dc.source_current = dc_current            # Sets the source current to 0 mA
    dc.enable_source() 
    v_up_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_1.append(float(volt.query("fetch?")))
    v_up_1 = sorted(v_up_1)
    v_up_1 = v_up_1[3: -3]
    
    
    v_up_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):  # apply positive current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_up_2.append(float(volt.query("fetch?")))
    v_up_2 = sorted(v_up_2)
    v_up_2 = v_up_2[3: -3]
    
    # Sets the compliance voltage to 10 V
    dc.source_current = -1 * dc_current
    dc.enable_source() 
    
    v_down_1 = []
    nano_change_chanel(volt, chan=1)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_1.append(float(volt.query("fetch?")))
        
        
    v_down_2 = []
    nano_change_chanel(volt, chan=2)
    for i in range(measure_points):   # apply negative current
        time.sleep(time_per_point)
        if i < skip_points:
            continue
        v_down_2.append(float(volt.query("fetch?")))
    
    dc.shutdown() 

    
    v_down_1 = sorted(v_down_1)
    v_down_1 = v_down_1[3: -3]
    v_down_2 = sorted(v_down_2)
    v_down_2 = v_down_2[3: -3]
    
    average_v_1 = (np.array(v_up_1).mean() - np.array(v_down_1).mean())/2
    std_v_1 = (np.array(v_up_1).std() + np.array(v_down_1).std())/2
    
    average_v_2 = (np.array(v_up_2).mean() - np.array(v_down_2).mean())/2
    std_v_2 = (np.array(v_up_2).std() + np.array(v_down_2).std())/2
    return average_v_1, std_v_1, average_v_2, std_v_2

import subprocess
def get_position():
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_position.py'])

    current_p = round(float(result.decode('utf-8').strip().split(',')[1][:-1]))
    return current_p

def set_T(t=300):
    if t < 2 or t > 400:
        raise
    result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms.py', str(t)])

    current_t = result.decode('utf-8').strip()
    if abs(float(current_t)-t)<1:
        print("current T {}".format(current_t))
        return
    else:
        print("set t:")
        print(t)
        print("current t:")
        print(current_t)
        raise
        
def set_Field(field=0, persistent=False):
    if abs(field) > 140000:
        raise
    if persistent:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                      r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py',
                                          str(field), str('persistent')])
    else:
        result = subprocess.check_output([r'C:\Program Files\IronPython 2.7\ipy.exe', 
                                          r'C:\Users\yanglabuser\Desktop\Sam files\_switch\ppms-master\ppms_set_field.py', 
                                          str(field)])

    current_field = result.decode('utf-8').strip()
    print(current_field)
    if abs(float(current_field)-field)<2:
        print("current field {}".format(current_field))
        return
    else:
        print("set field:")
        print(field)
        print("current t:")
        print(current_field)
        raise
        
def generate_fields(fileds, steps):
    assert (len(fileds) == len(steps))
    res = []
    for i in range(len(fileds)):
        res.extend([f for f in range(fileds[i], fileds[i + 1] if i+1 < len(fileds) else 0, 
                                   -1 * steps[i])])
    res = res + [0] + [-1* f for f in res[::-1]]
    res += res[::-1][1:]
    return res  

def field_scan_two_sample_two_channel(dc1, volt1, dc2, volt2, dc_current_1=1e-4, dc_current_2=1e-4, fields=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        file_2 = "/data/" + sample2_id + "_" + str(T) + "K_" + str(int(dc_current_2*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        with open(file_2, 'w') as f:
            f.write("field(G), average_v_1, std_v_1,  average_v_2, std_v_2, dc_current\n")
        dc_current = None
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            for file in [file_1, file_2]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc1, volt=volt1, dc_current=dc_current)
                else:
                    dc_current = dc_current_2
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc2, volt=volt2, dc_current=dc_current)
                    
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, dc_current))

rm = visa.ResourceManager()
rm.list_resources()

('GPIB0::1::INSTR',
 'GPIB0::6::INSTR',
 'GPIB0::7::INSTR',
 'GPIB0::9::INSTR',
 'GPIB0::12::INSTR',
 'GPIB0::22::INSTR',
 'GPIB0::23::INSTR')

In [3]:
def setcurr(curr=1, count_write=50, thrd=0.01):    
    for i in range(count_write):
            magnet.write("CURR {}".format(curr))
            time.sleep(0.1)
    print ('Set current done.', curr)

def covert_to_float(s):
    ans = ''
    for i in range(len(s) - 2):
        ans += s[i]
        if s[i+1].isalpha() & s[i+2].isalpha(): break
    return float(ans)

def fieldmeasure():
    s = gauss.query(":FETCH?")
    field=covert_to_float(s)*10000
    return field

In [2]:
def field_scan_one_sample_two_channel(dc1, volt1, dc_current_1=1e-4, fields=[], Ts=[None], sample1_id="1", sample2_id="2",
                                      t_wait=10, field_wait=5):
    for T in Ts:
        if not T is None:
            set_T(T)
            print(datetime.datetime.now())
            time.sleep(t_wait*60)
        else:
            T = "300"
            
        file_1 = "data/" + sample1_id + "_" + str(T) + "K_" + str(int(dc_current_1*1e6)) + "uA"+ str(int(time.time())) + ".csv"
        
        with open(file_1, 'w') as f:
            f.write("field(G), average_v_1, std_v_1, average_v_2, std_v_2, dc_current\n")
            
        
        dc_current = None
        for field in fields:
            set_Field(field)
            time.sleep(field_wait)
            for file in [file_1]:
                if file == file_1:
                    dc_current = dc_current_1
                    average_v_1, std_v_1, average_v_2, std_v_2 = measure_resistance_two_channel(dc=dc1, volt=volt1, dc_current=dc_current)
                   
                with open(file, 'a') as f:
                    f.write("{},{},{},{},{},{}\n".format(field, average_v_1, std_v_1, average_v_2, std_v_2, dc_current))

In [2]:
set_Field(0)

CalledProcessError: Command '['C:\\Program Files\\IronPython 2.7\\ipy.exe', 'C:\\Users\\yanglabuser\\Desktop\\Sam files\\_switch\\ppms-master\\ppms_set_field.py', '0']' returned non-zero exit status 1.

In [3]:
#hall measurement connection, chan 1 rxy, chan 2 rxx
dc1_ID=30
dc2_ID=30
volt1_ID=8
volt2_ID=8
dc1 = Keithley2400("GPIB::{}".format(dc1_ID))
dc2 = None #Keithley2400("GPIB::{}".format(dc2_ID))
volt1 = rm.open_resource('GPIB0::{}::INSTR'.format(volt1_ID))
volt2 = None # rm.open_resource('GPIB0::{}::INSTR'.format(volt2_ID))

#volt2.write("*RST")
#volt1.write("*RST")

In [4]:
volt1.write("*RST")

(6, <StatusCode.success: 0>)

In [5]:
init_hall_two_channel(step=2)

sample 1:0chan1, std_1, chan_2,std_2
(-3.893265371428572e-05, 1.252359435325567e-08, 0.03655350789285715, 5.824525915656977e-07)
sample 1:1chan1, std_1, chan_2,std_2
(-3.8922742310714285e-05, 1.052031333027571e-08, 0.0365523015, 1.1542958801404442e-07)
----------
sample 2:0chan1, std_1, chan_2,std_2


AttributeError: 'NoneType' object has no attribute 'source_current_range'

In [6]:
fs = [140000]
steps = [5000]
fields = generate_fields(fs, steps)
print(fields)
temperatures = [355, 325, 300]
sample1_id='Xd806_1p5nmPt3nmnmTa1nmPt'
sample2_id='2'

[140000, 135000, 130000, 125000, 120000, 115000, 110000, 105000, 100000, 95000, 90000, 85000, 80000, 75000, 70000, 65000, 60000, 55000, 50000, 45000, 40000, 35000, 30000, 25000, 20000, 15000, 10000, 5000, 0, -5000, -10000, -15000, -20000, -25000, -30000, -35000, -40000, -45000, -50000, -55000, -60000, -65000, -70000, -75000, -80000, -85000, -90000, -95000, -100000, -105000, -110000, -115000, -120000, -125000, -130000, -135000, -140000, -135000, -130000, -125000, -120000, -115000, -110000, -105000, -100000, -95000, -90000, -85000, -80000, -75000, -70000, -65000, -60000, -55000, -50000, -45000, -40000, -35000, -30000, -25000, -20000, -15000, -10000, -5000, 0, 5000, 10000, 15000, 20000, 25000, 30000, 35000, 40000, 45000, 50000, 55000, 60000, 65000, 70000, 75000, 80000, 85000, 90000, 95000, 100000, 105000, 110000, 115000, 120000, 125000, 130000, 135000, 140000]


In [ ]:
field_scan_two_sample_two_channel(dc1, volt1, dc2, volt2, dc_current_1=1e-4, dc_current_2=1e-4, 
                                  fields=fields, Ts=temperatures, sample1_id=sample1_id, sample2_id=sample2_id,
                                      t_wait=10, field_wait=5)
#set_Field(field=0, persistent=True)

In [7]:
field_scan_one_sample_two_channel(dc1, volt1, dc_current_1=5e-5, fields=fields, Ts=temperatures, sample1_id=sample1_id, sample2_id="2",
                                      t_wait=10, field_wait=5)


set_Field(0, True)

current T 355.0863
2020-03-10 18:18:29.417715
140000.09375
current field 140000.09375
134999.8125
current field 134999.8125
129999.742188
current field 129999.742188
124999.929688
current field 124999.929688
119999.117188
current field 119999.117188
114999.742188
current field 114999.742188
109999.96875
current field 109999.96875
104999.421875
current field 104999.421875
99999.9453125
current field 99999.9453125
94999.671875
current field 94999.671875
89999.8828125
current field 89999.8828125
84999.4296875
current field 84999.4296875
79999.7578125
current field 79999.7578125
74999.7421875
current field 74999.7421875
69999.8125
current field 69999.8125
64999.7382813
current field 64999.7382813
59999.6445313
current field 59999.6445313
54999.6445313
current field 54999.6445313
49999.7226563
current field 49999.7226563
44999.7265625
current field 44999.7265625
39999.8046875
current field 39999.8046875
34999.78125
current field 34999.78125
29999.6855469
current field 29999.6855469
24999.81

-24999.7128906
current field -24999.7128906
-19999.8398438
current field -19999.8398438
-14999.8427734
current field -14999.8427734
-9999.66992188
current field -9999.66992188
-4999.89746094
current field -4999.89746094
0.0250031482428
current field 0.0250031482428
4999.94775391
current field 4999.94775391
10000.0449219
current field 10000.0449219
15000.1171875
current field 15000.1171875
20000.0644531
current field 20000.0644531
25000.1367188
current field 25000.1367188
29999.9863281
current field 29999.9863281
34999.984375
current field 34999.984375
40000.1289063
current field 40000.1289063
45000.2265625
current field 45000.2265625
50000.2265625
current field 50000.2265625
55000.171875
current field 55000.171875
60000.2421875
current field 60000.2421875
65000.1679688
current field 65000.1679688
70000.2890625
current field 70000.2890625
75000.234375
current field 75000.234375
80000.2890625
current field 80000.2890625
85000.109375
current field 85000.109375
90000.2109375
current field 

In [12]:
set_Field(0, True)

-0.0225028004497
current field -0.0225028004497


In [13]:
set_T(300)

current T 300.0582
